#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import streamlit as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy


#### Load Datasets

In [19]:
print(tf.__version__)

2.10.0


In [15]:
df_1 = pd.read_csv("dataset.csv")

print("Dataset shape is:", df_1.shape)

df_1.head()

Dataset shape is: (4925, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acne,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Acne,skin_rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Acne,skin_rash,pus_filled_pimples,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
def uniq_symp(df):
    a = df["Symptom_1"].unique().tolist()
    b = df["Symptom_2"].unique().tolist()
    c = df["Symptom_3"].unique().tolist()
    d = df["Symptom_4"].unique().tolist()
    e = df["Symptom_5"].unique().tolist()
    f = df["Symptom_6"].unique().tolist()
    g = df["Symptom_7"].unique().tolist()
    h = df["Symptom_8"].unique().tolist()
    i = df["Symptom_9"].unique().tolist()
    j = df["Symptom_10"].unique().tolist()
    k = df["Symptom_11"].unique().tolist()
    l = df["Symptom_12"].unique().tolist()
    m= df["Symptom_13"].unique().tolist()
    n = df["Symptom_14"].unique().tolist()
    o = df["Symptom_15"].unique().tolist()
    p = df["Symptom_16"].unique().tolist()
    q= df["Symptom_17"].unique().tolist()



    z = a+b+c+d+e+f+g+h+i+j+k+l+m+o+p+q
    symp = {"Symptoms": z}
    df = pd.DataFrame(symp, index=range(len(z)))
    uniq = df["Symptoms"].unique().tolist()

    return uniq

In [2]:
df_2 = pd.read_csv("symptom_Description.csv")

print("Description shape is:", df_2.shape)

df_2.head()

Description shape is: (42, 2)


,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [4]:
df_3 = pd.read_csv("symptom_precaution.csv")

print("Precaution shape is:", df_3.shape)
#df_3.fillna()
df_3.head()

Precaution shape is: (41, 5)


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug_Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,use ice to compress itching,know and avoid allergens
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [3]:
"""A function that takes in a disease name , gives you a discription about the disease and precautions to take."""
def drug_info(disease):
    d = disease

    # Print disease description
    print("The predicted disease is:", d.strip().replace("_", " "))
    print(" ")
    print("Description of", d.strip().replace("_", " "),":")
    print(df_2[df_2["Disease"] == d]["Description"].item())
    print(" ")

    # Print Precautions to take
    print("Precautions to take are:")
    print(df_3[df_3["Disease"] == d]["Precaution_1"].item().title()) 
    print(df_3[df_3["Disease"] == d]["Precaution_2"].item().title())
    print(df_3[df_3["Disease"] == d]["Precaution_3"].item().title())
    print(df_3[df_3["Disease"] == d]["Precaution_4"].item().title())

In [14]:
drug_info("Malaria")

The predicted disease is: Malaria
 
Description of Malaria :
An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
 
Precautions to take are:
Consult Nearest Hospital
Avoid Oily Food
Avoid Non Veg Food
Keep Mosquitos Out


In [18]:
df_4 = pd.read_csv("Symptom-severity.csv")

print("Severity shape is:", df_4.shape)

df_4.head()

Severity shape is: (133, 2)


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [25]:
def encode_symptoms(df, df_4):
    for i in df_4.index:
        symptom = df_4["Symptom"][i]
        weight = df_4["weight"][i]
        df = df.replace(symptom, weight)

    # Replace missing values with 0
    df = df.fillna(0)

    # Additional hardcoded replacements
    df = df.replace("foul_smell_of_urine", 5)
    df = df.replace("dischromic__patches", 6)
    df = df.replace("spotting__urination", 6)
    
    return df

In [26]:
new_df_1 = encode_symptoms(df_1, df_4)
new_df_1.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,3,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acne,3,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acne,2,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acne,3,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acne,3,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_df = pd.read_csv("Training.csv")
test_df = pd.read_csv("Testing.csv")

In [31]:
list(train_df.columns)

['itching',
 'skin_rash',
 'nodal_skin_eruptions',
 'continuous_sneezing',
 'shivering',
 'chills',
 'joint_pain',
 'stomach_pain',
 'acidity',
 'ulcers_on_tongue',
 'muscle_wasting',
 'vomiting',
 'burning_micturition',
 'spotting_ urination',
 'fatigue',
 'weight_gain',
 'anxiety',
 'cold_hands_and_feets',
 'mood_swings',
 'weight_loss',
 'restlessness',
 'lethargy',
 'patches_in_throat',
 'irregular_sugar_level',
 'cough',
 'high_fever',
 'sunken_eyes',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'yellowish_skin',
 'dark_urine',
 'nausea',
 'loss_of_appetite',
 'pain_behind_the_eyes',
 'back_pain',
 'constipation',
 'abdominal_pain',
 'diarrhoea',
 'mild_fever',
 'yellow_urine',
 'yellowing_of_eyes',
 'acute_liver_failure',
 'fluid_overload',
 'swelling_of_stomach',
 'swelled_lymph_nodes',
 'malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',


## Dropping the least significant parameters from the dataset

In [5]:
y_train=train_df['prognosis']
x_train=train_df.drop(['malaise',
 'blurred_and_distorted_vision',
 'throat_irritation',
 'redness_of_eyes',
 'runny_nose',
 'congestion',
 'chest_pain',
 'weakness_in_limbs',
 'fast_heart_rate',
 'pain_during_bowel_movements',
 'pain_in_anal_region',
 'bloody_stool',
 'irritation_in_anus',
 'neck_pain',
 'dizziness',
 'cramps',
 'bruising',
 'obesity',
 'swollen_legs',
 'puffy_face_and_eyes',
 'brittle_nails',
 'swollen_extremeties',
 'excessive_hunger',
 'extra_marital_contacts',
 'drying_and_tingling_lips',
 'knee_pain',
 'hip_joint_pain',
 'muscle_weakness',
 'stiff_neck',
 'swelling_joints',
 'movement_stiffness',
 'spinning_movements',
 'loss_of_balance',
 'unsteadiness',
 'weakness_of_one_body_side',
 'loss_of_smell',
 'bladder_discomfort',
 'foul_smell_of urine',
 'continuous_feel_of_urine',
 'passage_of_gases',
 'depression',
 'irritability',
 'muscle_pain',
 'red_spots_over_body',
 'belly_pain',
 'abnormal_menstruation',
 'dischromic _patches',
 'watering_from_eyes',
 'increased_appetite',
 'polyuria',
 'mucoid_sputum',
 'lack_of_concentration',
 'visual_disturbances',
 'receiving_blood_transfusion',
 'stomach_bleeding',
 'distention_of_abdomen',
 'history_of_alcohol_consumption',
 'blood_in_sputum',
 'prominent_veins_on_calf',
 'palpitations',
 'painful_walking',
 'pus_filled_pimples',
 'blackheads',
 'scurring',
 'skin_peeling',
 'small_dents_in_nails',
 'inflammatory_nails',
 'blister',
 'red_sore_around_nose',
 'prognosis'],axis=1)


In [13]:
x_test=test_df.drop(['malaise',
 'blurred_and_distorted_vision',
 'phlegm',
 'throat_irritation',
 'redness_of_eyes',
 'sinus_pressure',
 'runny_nose',
 'congestion',
 'chest_pain',
 'weakness_in_limbs',
 'fast_heart_rate',
 'pain_during_bowel_movements',
 'pain_in_anal_region',
 'bloody_stool',
 'irritation_in_anus',
 'neck_pain',
 'dizziness',
 'cramps',
 'bruising',
 'obesity',
 'swollen_legs',
 'swollen_blood_vessels',
 'puffy_face_and_eyes',
 'enlarged_thyroid',
 'brittle_nails',
 'swollen_extremeties',
 'excessive_hunger',
 'extra_marital_contacts',
 'drying_and_tingling_lips',
 'slurred_speech',
 'knee_pain',
 'hip_joint_pain',
 'muscle_weakness',
 'stiff_neck',
 'swelling_joints',
 'movement_stiffness',
 'spinning_movements',
 'loss_of_balance',
 'unsteadiness',
 'weakness_of_one_body_side',
 'loss_of_smell',
 'bladder_discomfort',
 'foul_smell_of urine',
 'continuous_feel_of_urine',
 'passage_of_gases',
 'internal_itching',
 'toxic_look_(typhos)',
 'depression',
 'irritability',
 'muscle_pain',
 'altered_sensorium',
 'red_spots_over_body',
 'belly_pain',
 'abnormal_menstruation',
 'dischromic _patches',
 'watering_from_eyes',
 'increased_appetite',
 'polyuria',
 'family_history',
 'mucoid_sputum',
 'rusty_sputum',
 'lack_of_concentration',
 'visual_disturbances',
 'receiving_blood_transfusion',
 'receiving_unsterile_injections',
 'coma',
 'stomach_bleeding',
 'distention_of_abdomen',
 'history_of_alcohol_consumption',
 'fluid_overload.1',
 'blood_in_sputum',
 'prominent_veins_on_calf',
 'palpitations',
 'painful_walking',
 'pus_filled_pimples',
 'blackheads',
 'scurring',
 'skin_peeling',
 'silver_like_dusting',
 'small_dents_in_nails',
 'inflammatory_nails',
 'blister',
 'red_sore_around_nose',
 'yellow_crust_ooze',
 'prognosis'],axis=1)
y_test=test_df['prognosis']

In [6]:
x_train.columns

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition',
       'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety',
       'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness',
       'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough',
       'high_fever', 'sunken_eyes', 'breathlessness', 'sweating',
       'dehydration', 'indigestion', 'headache', 'yellowish_skin',
       'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes',
       'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea',
       'mild_fever', 'yellow_urine', 'yellowing_of_eyes',
       'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach',
       'swelled_lymph_nodes'],
      dtype='object')

### Data Standardization

In [7]:
le=LabelEncoder()
dummy=le.fit_transform(train_df['prognosis'])
y_train=pd.DataFrame(dummy)
y_train.head()

,0
0,15
1,15
2,15
3,15
4,15


## Listing the indices of diseases

In [8]:
disease_classes = le.classes_ 
disease_classes

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes ',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'No disease', 'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [9]:
dummy=LabelEncoder().fit_transform(test_df['prognosis'])
y_test=pd.DataFrame(dummy)

In [10]:
model=Sequential([
    Dense(units=200,activation='relu'),
    Dense(units=150,activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=42,activation='softmax')
])

In [14]:
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])
# model.summary

In [15]:
history=model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test))

Epoch 1/10


c:\Users\ThisPC\anaconda3\envs\tf-new\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


37/39 [===========================>..] - ETA: 0s - loss: 2.9828 - accuracy: 0.4736

c:\Users\ThisPC\anaconda3\envs\tf-new\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "c:\Users\ThisPC\anaconda3\envs\tf-new\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ThisPC\anaconda3\envs\tf-new\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 972, in launch_instance
      app.start()
    File "C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\ThisPC\

In [16]:
prediction=model.predict(x_test)
prediction[1]

2/2 [==============================] - 0s 4ms/step


array([0.00765537, 0.02403797, 0.00938307, 0.0017924 , 0.50035495,
       0.0062532 , 0.00951521, 0.00450885, 0.00467874, 0.00891396,
       0.01875634, 0.00790946, 0.00752461, 0.00501935, 0.00736558,
       0.01292176, 0.00530887, 0.02107194, 0.00731683, 0.0027801 ,
       0.01036113, 0.00582967, 0.00229355, 0.01358688, 0.0253823 ,
       0.00910818, 0.01206752, 0.01042092, 0.01335816, 0.01285807,
       0.02050958, 0.00896003, 0.01813376, 0.0091944 , 0.00744078,
       0.03607876, 0.01430523, 0.04806309, 0.02719125, 0.00647448,
       0.00518243, 0.01013124], dtype=float32)

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'], label='train')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.show()

# plt.plot(history.history['accuracy'])
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.show()


In [ ]:
pred=np.argmax(prediction, axis=1)
original=y_test[0]
print(original[41])

In [ ]:
# confusion = confusion_matrix(original,pred)

# fig = px.imshow(confusion, labels=dict(x="Predicted Value", y="Actual Vlaue"),text_auto=True, title='Confusion Matrix')
# fig.update_layout(title_x=0.5)
# fig.show()

In [ ]:
count=0
for i in range(41):
    if(pred[i]-original[i]!=0):
        count+=1

Test_accuracy=(42-count)/42*100
for i in range(10):
    print(f"Predicted: {pred[i]}, Actual: {original[i]}")

In [ ]:
pred_ga=np.argmax(ga, axis=1)
pred_ga[0]

####  Random Forest Classifier

In [17]:
clf = RandomForestClassifier(n_estimators=100, random_state=42) 
clf.fit(x_train, np.ravel(y_train))

RandomForestClassifier(random_state=42)

In [18]:
rf_predictions = clf.predict(x_test)
print(len(rf_predictions))

43


In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, rf_predictions)
mse = mean_squared_error(y_test, rf_predictions)
r2 = r2_score(y_test, rf_predictions)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

ValueError: could not convert string to float: 'Fungal infection'

In [20]:
print(f"Random Forest Test Accuracy:",Test_accuracy,'%')

NameError: name 'Test_accuracy' is not defined

In [21]:
count=0
for i in range(41):
    if(rf_predictions[i]-original[i]!=0):
        count+=1

rf_Test_accuracy=(42-count)/42*100

NameError: name 'original' is not defined

## Saving the Model

In [23]:
from joblib import dump,load

In [ ]:
# filename = 'rf_model.joblib'
# model = clf
# dump(model, filename)

In [24]:
ml_model=load('rf_model.joblib')

In [26]:
inp = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Reshape the 1D array into a 2D array with 1 row and 48 columns
inp = inp.reshape(1,-1)

# Make the prediction
prediction = ml_model.predict(inp)
print(prediction)

[5]


C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
da_list = da.iloc[0].to_list()
da_arr = np.array(da_list)
da_arr = da_arr.reshape(1,-1)

# Make the prediction
prediction = ml_model.predict(da_arr)
prediction
dis = prediction[0]
disease_classes[dis]

### Initial Random Forest based on encoding disease severity

In [ ]:
def encode_symptoms(df, data_severity):
    for i in data_severity.index:
        symptom = data_severity["Symptom"][i]
        weight = data_severity["weight"][i]
        df = df.replace(symptom, weight)

    # Replace missing values with 0
    df = df.fillna(0)

    # Additional hardcoded replacements
    df = df.replace("foul_smell_of_urine", 5)
    df = df.replace("dischromic__patches", 6)
    df = df.replace("spotting__urination", 6)
    
    return df

In [ ]:
new_df = encode_symptoms(df, data_severity)

In [16]:
# separating the data and labels
X = new_df.drop(columns='Disease', axis=1)
Y = new_df['Disease']

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X)

In [ ]:
X = scaler.transform(X)

In [133]:
from sklearn.ensemble import RandomForestClassifier
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rfc_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rfc_classifier.fit(X_train, Y_train)

# Predict disease labels on the testing data
Y_pred = rfc_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)

precision = precision_score(Y_test, Y_pred, average='macro')
recall = recall_score(Y_test, Y_pred,average='macro')
f1 = f1_score(Y_test, Y_pred, average='macro')

print("Random Forest")
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(accuracy_score(Y_test, Y_pred,normalize=False))
print("Confusion matrix")
conf_matrix=confusion_matrix(Y_test,Y_pred)
print(conf_matrix)

Random Forest
Accuracy: 0.9908536585365854
Precision: 0.9916531165311654
Recall: 0.9918172648227722
F1-Score: 0.9914105369307706
975
Confusion matrix
[[31  0  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 22  0]
 [ 0  0  0 ...  0  0 24]]


In [ ]:
"""
A function that takes in a list of numbers corresponsing to the symptom. Returns a dataframe suitable to undergo prediction
"""
def dis_data(values):
    symp_keys = x_train.columns
    symp_values = values
    dic = {symp_keys[i]: symp_values[i] for i in range(len(symp_keys))}
    df = pd.DataFrame(list(dic.items())).transpose()
    df.columns = df.iloc[0]
    df.drop(index=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns="index", inplace=True)

    return df

In [ ]:
d = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
da = dis_data(d)
da["itching"] = 1
#g = da.iloc[0].to_list()
ga = np.asarray(da).astype(np.float32)
model.predict(ga)
#da.to_csv("empty_df.csv")